In [20]:
import  dash 
from santiago_core import SANTIAGO_CORE
from dash import html ,dcc
import pandas as pd 
import plotly.express as px
import numpy as np
from re import split
import constants as const
import datetime 
pd.set_option('display.max_columns', 500)

In [8]:
alertas = pd.read_excel('input/220303_consolidado_alertas_nc.xlsx')

In [9]:
sa = SANTIAGO_CORE()

In [38]:
con = pd.read_csv('input/con_fixed.csv', dtype=object) # Consolidado con arreglos 

In [70]:
ep = pd.read_excel('input/211020_empleados_planta.xlsx', dtype=str) 
ep['nombre_completo'] = ep['Apell_Paterno'] + ' ' +  ep['Apell_Materno'] + ' ' + ep['Nombre']  
ep = ep[const.cols_emp_planta]

In [71]:
df_con_ep = con.merge(ep, how='left', left_on=['Cvendedor'], right_on=['Cod_Empleado'])

Calcular y graficar cantidad de días entre ventas y NC


In [73]:

df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('ene', '01')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('feb', '02')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('mar', '03')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('abr', '04')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('may', '05')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('jun', '06')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('jul', '07')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('ago', '08')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('sep', '09')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('oct', '10')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('nov', '11')
df_con_ep.Dcompra_ant = df_con_ep.Dcompra_ant.str.replace('dic', '12')
df_con_ep.Dcompra_ant = pd.to_datetime(df_con_ep.Dcompra_ant, format='%d-%m-%Y') # TODO arreglar para español +


In [74]:
df_con_ep.Dcompra_nvo = pd.to_datetime(df_con_ep.Dcompra_nvo, format='%Y-%m-%d') # TODO arreglar para español 


In [77]:
df_con_ep["dif_dias"] = (df_con_ep.Dcompra_nvo - df_con_ep.Dcompra_ant).dt.days


0          8
1         15
2          4
3         13
4         16
          ..
238132     6
238133    44
238134     3
238135     3
238136    10
Name: dif_dias, Length: 238137, dtype: int64

In [66]:
diferencia_dias = df_nc_ep

In [ ]:
mes=0
mes_sig=0
mes_ant=0
while mes_sig <=2302:
    mes = mes + 1
    mes_sig= mes_sig + 30
    mes_ant = mes_sig - 30
    diferencia_dias.loc[(diferencia_dias["dif_dias"] >= f"{str(mes_ant)} days") & ((diferencia_dias["dif_dias"] <= f"{str(mes_sig)} days")), "mes"] = f"mes {mes}"
    


In [79]:
diferencia_dias = df_con_ep.groupby("dif_dias")["Cautoriza"].nunique().reset_index()

In [80]:
diferencia_dias = diferencia_dias.loc[diferencia_dias.Cautoriza !=1 ]

In [83]:

dias_graf = px.bar(diferencia_dias, x="dif_dias", y="Cautoriza",title = "Diferencia de días entre venta y NCs",labels={"Cautoriza":"Cantidad","dif_dias":"Dias"})
dias_graf.update_layout(xaxis_categoryorder = 'total descending')
dias_graf.show()



In [ ]:
top_20_clientes_nc = sa.df_nc.groupby("Nrutcomprador")["Cautoriza"].nunique().reset_index().sort_values("Cautoriza",ascending=False)

In [ ]:
top_20_clientes_nc.loc[~top_20_clientes_nc.Nrutcomprador.isin(["0","1","2"])].head(20)

,Nrutcomprador,Cautoriza
50024,1234567890,38
95301,52295256,28
38604,1110555895,24
105150,63317421,22
107486,66710306,21
15405,1020805672,20
87759,459687456,18
72764,37723241,16
77710,41328972,16
124524,90113482,16


In [62]:
nc_r= sa.df_nc.loc[sa.df_nc.Nrutcomprador == "0" ] 
ced_0 = nc_r.groupby(["Desc_local","Dcompra_nvo"])["Cvendedor"].nunique().sort_values(ascending=False).reset_index()
graf_ced_0 = px.bar(ced_0, x="Dcompra_nvo", y="Cvendedor", color="Desc_local" )
graf_ced_0.show()

In [ ]:
nc_r= sa.df_nc.loc[sa.df_nc.Nrutcomprador == "1" ] 
ced_1 = nc_r.groupby(["Desc_local","Dcompra_nvo"])["Cvendedor"].nunique().sort_values(ascending=False).reset_index()
graf_ced_1 = px.bar(ced_1, x="Dcompra_nvo", y="Cvendedor", color="Desc_local" )
graf_ced_1.show()

In [ ]:
nc_r= sa.df_nc.loc[sa.df_nc.Nrutcomprador == "2" ] 
ced_1 = nc_r.groupby(["Desc_local","Dcompra_nvo"])["Cvendedor"].nunique().sort_values(ascending=False).reset_index()
graf_ced_2 = px.bar(ced_1, x="Dcompra_nvo", y="Cvendedor", color="Desc_local" )
graf_ced_2.show()

In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

C:\Users\ext_maperezr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

